In [1]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import re
import openpyxl
import time

In [2]:
#赛季列表
season_list=[]
for s_year in range(2000,2019):
    e_year=str(s_year+1)[-2:]
    season_list.append(str(s_year)+'-'+e_year)
print(season_list)


['2000-01', '2001-02', '2002-03', '2003-04', '2004-05', '2005-06', '2006-07', '2007-08', '2008-09', '2009-10', '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19']


In [3]:
#获取赛季总览页面
main_url='https://www.kicker.de'
tabelle_url='https://www.kicker.de/1-bundesliga/tabelle/'
headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'}
def get_html(main_url,tabelle_url,season_num):
    #headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'}
    data=requests.get(tabelle_url+season_num,headers=headers)
    element=etree.HTML(data.text)
    return element

In [4]:
#获取俱乐部url
def get_club_url(element):
    club_url=element.xpath('//td[@class="kick__table--ranking__teamname kick__table--ranking__index kick__t__a__l kick__respt-m-o-4 kick__respt-m-w-120 kick__t__a__l"]/a[1]/@href')
    for i in range(0,len(club_url)):
        club_url[i]=main_url+club_url[i].replace('/info/','/kader/')#将info替换为kader，并接在main_url后面
    return club_url
    

In [5]:
#获取球员url和所属俱乐部
def get_player_url(club):
    data=requests.get(club,headers=headers)
    element=etree.HTML(data.text)
    club_name=element.xpath('//title/text()')[0].split(' | ')[0]
    player_url=element.xpath('//td[@class="kick__table--ranking__index kick__t__a__l kick__respt-m-w-190"]/a/@href')
    for i in range(0,len(player_url)):
        player_url[i]=main_url+player_url[i]
    return club_name,player_url

In [6]:
#获取球员名字
def get_player_name(player):
    data=requests.get(player,headers=headers)
    time.sleep(0.5)
    soup=BeautifulSoup(data.text,'lxml')
    player_name=soup.title.get_text().split(' | ')[0]
    return soup,player_name


In [7]:
#获取球员基本信息
def get_player_info(soup,rank,club_name,player_name):
    p_info_content=soup.find('table',attrs={'class':'kick__table--per-data kick__table--per-info'})
    p_info_formal=str(p_info_content).replace(' ','').replace('\r','').replace('\n','')#去除空格和换行符
    #球员信息的正则表达式
    birth_re='Geborenam</td><td>([0-9]{2}.[0-9]{2}.[0-9]{4})<'
    pos_re='Position</td><td>(.+?)</td>'
    height_re='Größe</td><td>([0-9]{3})cm'
    weight_re='Gewicht</td><td>([0-9]+)kg'
    nation_re='png"/></span>(.+?)</td>'
    birth=re.compile(birth_re).findall(p_info_formal)
    pos=re.compile(pos_re).findall(p_info_formal)
    height=re.compile(height_re).findall(p_info_formal)
    weight=re.compile(weight_re).findall(p_info_formal)
    nation=re.compile(nation_re).findall(p_info_formal)
    p_info_1=[birth,pos,height,weight,nation]
    p_info_1=[p[0] if p !=[] else '' for p in p_info_1]
    p_info=[rank,club_name,player_name]
    p_info.extend(p_info_1)
    #p_info_sum.append(p_info)
    return p_info

In [8]:
#获取指定赛季的球员数据url
def get_tar_season(soup,season_num,main_url):
    tar_season=season_num.replace('-','/')
    season_select=soup.find('select',attrs={'class':'kick__head-dropdown__select'})
    season_select_formal=str(season_select).replace(' ','').replace('\r','').replace('\n','')
    season_re='value="([^<]+)">'+tar_season
    season_url=re.compile(season_re).findall(season_select_formal)
    tar_season_url=main_url+season_url[0]
    return tar_season_url
    

In [14]:
#获取球员比赛数据
def get_game_data(tar_season_url):
    data=requests.get(tar_season_url,headers=headers)
    soup=BeautifulSoup(data.text,'lxml')
    g_info=soup.find('tr',attrs={'class':'kick__js-open-saison-detail'})
    g_info_formal=str(g_info).replace(' ','').replace('\r','').replace('\n','')
    g_data_re='>([0-9,/-]+)</td'
    g_data=re.compile(g_data_re).findall(g_info_formal)
    return g_data

## 调度

In [18]:
main_url='https://www.kicker.de'
tabelle_url='https://www.kicker.de/1-bundesliga/tabelle/'
for season_num in season_list[-2,-1]:
    sum_info=[]
    element=get_html(main_url,tabelle_url,season_num)
    club_url=get_club_url(element)
    rank=1
    for club in club_url:
        club_name,player_url=get_player_url(club)
        #print(club_name)
        for player in player_url:
            soup,player_name=get_player_name(player)
            p_info=get_player_info(soup,rank,club_name,player_name)
            tar_season_url=get_tar_season(soup,season_num,main_url)
            g_data=get_game_data(tar_season_url)
            p_info.extend(g_data)
            #print(p_info[2])
            sum_info.append(p_info)
        rank+=1
print(len(sum_info))
print(sum_info[-1])            
        

Bayern München
FC Schalke 04
TSG Hoffenheim
Borussia Dortmund
Bayer 04 Leverkusen
RB Leipzig
VfB Stuttgart
Eintracht Frankfurt
Bor. Mönchengladbach
Hertha BSC
Werder Bremen
FC Augsburg
Hannover 96
1. FSV Mainz 05
SC Freiburg
VfL Wolfsburg
Hamburger SV
1. FC Köln
484
[18, '1. FC Köln', 'Simon Zoller', '26.06.1991', 'Sturm', '179', '73', 'Deutschland', '17/14', '4,25', '2', '0/0', '1', '3', '5', '8', '3', '0', '0']


## 调整插入Excel的数据格式

In [19]:
for p_info in sum_info:
    #print(p_info)
    for field in p_info:
        if p_info.index(field)==3:
            p_info[3]=int(field[-4:])
        elif (p_info.index(field)==5) & (field!=''):
            p_info[5]=int(field)
        elif (p_info.index(field)==6) & (field!=''):
            p_info[6]=int(field)
        elif p_info.index(field)==8:
            p_info[8]=field.split('/')
        elif (p_info.index(field)==9) & (field!='-'):
            p_info[9]=float(field.replace(',','.'))
        elif p_info.index(field)==11:
            p_info[11]=field.split('/')
print(sum_info[-1])

[18, '1. FC Köln', 'Simon Zoller', 1991, 'Sturm', 179, 73, 'Deutschland', ['17', '14'], 4.25, '2', ['0', '0'], '1', '3', '5', '8', '3', '0', '0']


## 插入excel

In [20]:
target_filepath='G:\研二\kicker数据 - 副本.xlsx'
data=openpyxl.load_workbook(target_filepath)
excel=data['2017-18']

In [21]:
i=2
for p_info in sum_info:
    excel.cell(row=i,column=1,value=p_info[0])
    excel.cell(row=i,column=2,value=p_info[1])
    excel.cell(row=i,column=3,value=p_info[2])
    excel.cell(row=i,column=4,value=p_info[3])
    excel.cell(row=i,column=5,value=p_info[4])
    excel.cell(row=i,column=6,value=p_info[5])
    excel.cell(row=i,column=7,value=p_info[6])
    excel.cell(row=i,column=8,value=p_info[7])
    excel.cell(row=i,column=9,value=p_info[8][0])
    excel.cell(row=i,column=10,value=p_info[8][1])
    excel.cell(row=i,column=11,value=p_info[9])
    excel.cell(row=i,column=12,value=p_info[10])
    excel.cell(row=i,column=13,value=p_info[11][0])
    excel.cell(row=i,column=14,value=p_info[11][1])
    excel.cell(row=i,column=15,value=p_info[12])
    excel.cell(row=i,column=16,value=p_info[13])
    excel.cell(row=i,column=17,value=p_info[14])
    excel.cell(row=i,column=18,value=p_info[15])
    excel.cell(row=i,column=19,value=p_info[16])
    excel.cell(row=i,column=20,value=p_info[17])
    excel.cell(row=i,column=21,value=p_info[18])
    i+=1

In [22]:
data.save(target_filepath)